# Problem Statement: Building Stock Portfolios

Create pools of stocks that move together on one or more fundamental financial metric. In simple words, the goal is to group stocks that exhibit similar patterns or trends in one or more fundamental financial metrics.

### Steps to create stock pools
1. Gather data: Collect historical data for the financial metrics that are choosen.
2. Preprocess data: Clean and normalize the data as needed.
3. Select similarity metric: Choose a suitable metric based on the requirements.
4. Apply clustering algorithm: Use the chosen algorithm to group the stocks based on their similarity in the selected metrics.
5. Analyze and interpret the results: Examine the resulting clusters to identify stocks with similar characteristics.

### Importing the Dataset
The purpose of this dataset to get the list of companies which are stored in the "Symbol" column. Once the dataset is imported, the company symbols are stored in a list.

In [13]:
# importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [14]:
# configuring the notebook
pd.set_option("display.max_columns", None)
sns.set_theme(style = "whitegrid")
warnings.filterwarnings("ignore")

In [15]:
# importing dataset
df_stocks = pd.read_csv("ind_nifty50list.csv")
df_stocks.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024


In [16]:
# storing the symbols in a list
symbols_list = list(df_stocks["Symbol"])
symbols_list

['ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BPCL',
 'BHARTIARTL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDUNILVR',
 'HDFC',
 'ICICIBANK',
 'ITC',
 'IOC',
 'INDUSINDBK',
 'INFY',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NTPC',
 'NESTLEIND',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBILIFE',
 'SHREECEM',
 'SBIN',
 'SUNPHARMA',
 'TCS',
 'TATACONSUM',
 'TATAMOTORS',
 'TATASTEEL',
 'TECHM',
 'TITAN',
 'UPL',
 'ULTRACEMCO',
 'WIPRO']

Once the symbols are extracted into a list, a `.NS` is concatenated at the end of every symbol to fetch the data from Yahoo Finance.

In [17]:
yf_symbols = list(map(lambda x: x + ".NS", symbols_list))
yf_symbols

['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BPCL.NS',
 'BHARTIARTL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'HDFC.NS',
 'ICICIBANK.NS',
 'ITC.NS',
 'IOC.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NTPC.NS',
 'NESTLEIND.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SHREECEM.NS',
 'SBIN.NS',
 'SUNPHARMA.NS',
 'TCS.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TECHM.NS',
 'TITAN.NS',
 'UPL.NS',
 'ULTRACEMCO.NS',
 'WIPRO.NS']

Defining a few variable/ financial metrics, and fetching the corresponding data from `yfinance` for the companies in the list.

In [18]:
import yfinance as yf

stock_financials = {
    "marketCap": [],
    "regularMarketVolume": [],
    "earningsQuarterlyGrowth": [],
    "bookValue": [],
    "totalRevenue": [],
    "returnOnAssets": [],
    "profitMargins": [],
    "earningsGrowth": []
}

for ticker in yf_symbols:
    stock_info = yf.Ticker(ticker).info
    stock_financials["marketCap"].append(stock_info.get("marketCap", None))
    stock_financials["regularMarketVolume"].append(stock_info.get("regularMarketVolume", None))
    stock_financials["earningsQuarterlyGrowth"].append(stock_info.get("earningsQuarterlyGrowth", None))
    stock_financials["bookValue"].append(stock_info.get("bookValue", None))
    stock_financials["totalRevenue"].append(stock_info.get("totalRevenue", None))
    stock_financials["returnOnAssets"].append(stock_info.get("returnOnAssets", None))
    stock_financials["profitMargins"].append(stock_info.get("profitMargins", None))
    stock_financials["earningsGrowth"].append(stock_info.get("earningsGrowth", None))

In [19]:
df = pd.DataFrame(stock_financials)
df.head()

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth
0,3.101745e+12,1621158.0,0.472,244.328,2.741933e+11,NaN,0.33220,0.472
1,3.145921e+12,1271772.0,-0.245,195.250,3.522494e+11,NaN,0.14421,-0.246
2,3.621931e+12,7700448.0,0.057,534.043,6.497045e+11,0.01887,0.41144,0.050
3,3.036221e+12,193054.0,0.181,1037.411,4.776608e+11,NaN,0.16761,0.198
4,4.544633e+12,878729.0,0.138,1241.009,3.295906e+11,NaN,0.45287,0.115


In [20]:
df.shape

(50, 8)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   marketCap                49 non-null     float64
 1   regularMarketVolume      49 non-null     float64
 2   earningsQuarterlyGrowth  48 non-null     float64
 3   bookValue                49 non-null     float64
 4   totalRevenue             49 non-null     float64
 5   returnOnAssets           11 non-null     float64
 6   profitMargins            49 non-null     float64
 7   earningsGrowth           47 non-null     float64
dtypes: float64(8)
memory usage: 3.3 KB


### Downloading Adjusted Closing Price (Adj Close)
It is a stock price that has been adjusted to reflect the effects of corporate actions such as stock splits, dividends, and rights offerings. It provides a more accurate representation of stock's historical performance by accounting for factors that can artifically inflate or deflate the closing price.

In [22]:
stock_prices = yf.download(yf_symbols, start = "2020-01-01")["Adj Close"]
stock_prices.columns = symbols_list

[*********************100%***********************]  50 of 50 completed

1 Failed download:
['HDFC.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [23]:
stock_prices.shape

(1161, 50)

In [24]:
stock_prices.tail()

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,BHARTIARTL,BRITANNIA,CIPLA,COALINDIA,DIVISLAB,DRREDDY,EICHERMOT,GRASIM,HCLTECH,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,MARUTI,NTPC,NESTLEIND,ONGC,POWERGRID,RELIANCE,SBILIFE,SHREECEM,SBIN,SUNPHARMA,TCS,TATACONSUM,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-03 00:00:00+00:00,1479.550049,3154.850098,1191.599976,11043.650391,1865.599976,7353.799805,1560.599976,355.399994,5916.049805,1653.199951,519.599976,5066.000000,6853.850098,4865.799805,2706.850098,1790.449951,NaN,1637.349976,758.750000,5646.500000,678.900024,2794.300049,1247.699951,1436.699951,1941.250000,176.130005,509.399994,930.950012,1783.800049,3690.149902,2784.850098,12397.099609,2530.750000,406.399994,322.200012,334.649994,3018.250000,1928.650024,824.799988,25507.599609,1811.500000,1199.000000,1085.099976,152.149994,4512.350098,1644.099976,3621.149902,11460.799805,602.200012,536.049988
2024-09-04 00:00:00+00:00,1472.099976,3231.649902,1177.699951,10963.700195,1871.900024,7299.500000,1561.900024,357.250000,5926.549805,1651.900024,503.649994,5096.600098,6787.200195,4833.549805,2757.649902,1785.250000,NaN,1641.800049,758.799988,5683.750000,666.799988,2841.250000,1236.349976,1434.400024,1922.449951,177.029999,506.350006,933.049988,1783.800049,3650.800049,2749.600098,12336.250000,2534.750000,405.100006,314.399994,332.899994,3029.100098,1912.199951,816.500000,25770.750000,1832.849976,1194.949951,1080.449951,151.179993,4479.250000,1645.300049,3607.699951,11580.450195,607.849976,519.150024
2024-09-05 00:00:00+00:00,1465.550049,3238.250000,1180.550049,10855.750000,1864.949951,7244.899902,1547.199951,360.700012,5850.000000,1627.750000,497.000000,5120.899902,6695.750000,4800.100098,2741.600098,1790.550049,NaN,1645.449951,751.849976,5734.200195,669.950012,2838.449951,1235.949951,1422.900024,1933.150024,181.339996,511.200012,925.450012,1777.949951,3624.149902,2723.100098,12298.599609,2504.899902,403.250000,311.399994,331.250000,2985.949951,1907.849976,818.750000,25662.250000,1826.500000,1188.650024,1069.150024,151.720001,4475.950195,1639.500000,3722.850098,11542.650391,618.700012,524.849976
2024-09-06 00:00:00+00:00,1442.400024,3273.699951,1158.750000,10830.099609,1857.150024,7317.149902,1539.250000,352.149994,5843.549805,1611.050049,488.549988,5137.799805,6667.149902,4752.500000,2692.850098,1756.099976,NaN,1636.949951,738.400024,5743.750000,667.099976,2838.949951,1208.150024,1409.349976,1901.849976,176.639999,501.700012,932.799988,1764.150024,3574.750000,2698.100098,12186.150391,2503.199951,394.799988,308.799988,329.799988,2929.649902,1896.300049,782.500000,25610.449219,1824.550049,1173.849976,1049.349976,151.220001,4456.750000,1623.250000,3695.500000,11432.849609,609.799988,520.599976
2024-09-09 00:00:00+00:00,1435.900024,3280.750000,1170.849976,10847.599609,1860.449951,7347.450195,1542.650024,347.799988,5939.450195,1620.150024,484.450012,5173.500000,6655.899902,4747.350098,2710.350098,1746.750000,NaN,1646.500000,735.799988,5745.299805,658.549988,2921.800049,1235.000000,1428.099976,1894.650024,175.339996,511.750000,934.250000,1790.150024,3578.300049,2708.850098,12145.750000,2516.750000,389.850006,298.899994,328.549988,2924.899902,1901.750000,784.250000,25798.599609,1821.849976,1192.050049,1038.699951,149.470001,4449.549805,1579.199951,3684.550049,11494.349609,604.400024,514.849976


In [25]:
# extracting stock_prices only from the year 2020
prices_2020 = stock_prices.loc["2020-01-01 00:00:00": "2020-12-31 00:00:00"]

# Calculating the percentage growth of the stock price from 2020-01-02 to 2020-08-04
stock_prices.loc["returns_2020"] = (prices_2020.loc["2020-08-04 00:00:00"])/ (prices_2020.loc["2020-01-02 00:00:00"] - 1) * 100
stock_prices

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BPCL,BHARTIARTL,BRITANNIA,CIPLA,COALINDIA,DIVISLAB,DRREDDY,EICHERMOT,GRASIM,HCLTECH,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,MARUTI,NTPC,NESTLEIND,ONGC,POWERGRID,RELIANCE,SBILIFE,SHREECEM,SBIN,SUNPHARMA,TCS,TATACONSUM,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,364.348297,1726.790894,745.452026,2703.128906,935.670654,4160.838867,436.938965,185.158905,2751.210938,458.291077,131.486282,1745.139282,2790.024902,2115.296387,723.675842,486.583893,NaN,1227.428589,613.773560,2121.393311,207.269104,1802.972290,518.854004,1433.394653,663.080322,56.667076,193.003799,249.207779,1669.007446,1195.268555,508.507751,7020.506836,1403.387207,93.288536,94.034668,82.895226,1356.042725,967.813721,314.799194,19948.822266,412.528473,311.602112,183.311218,18.817579,1980.292847,655.840759,1128.044189,3967.757324,556.150818,242.605316
2020-01-02 00:00:00+00:00,369.654572,1724.335449,753.666260,2678.157715,948.356201,4175.342773,438.770386,183.614792,2763.656006,455.979980,130.958954,1751.903687,2775.974854,2069.786377,746.921204,487.945038,NaN,1235.252441,626.415466,2118.689209,212.927170,1804.368774,522.575623,1476.561523,661.145508,57.139484,194.422333,256.969391,1666.514893,1227.524170,511.587585,7037.934570,1398.667847,93.173409,94.477348,82.725616,1379.128662,960.825623,319.364227,20844.416016,413.145935,311.121033,192.553802,19.505510,1971.203003,659.240051,1128.874756,4143.256348,562.957886,243.192963
2020-01-03 00:00:00+00:00,369.027466,1686.539062,739.726990,2636.153564,932.418274,4123.619629,438.673981,182.089539,2750.441406,452.561279,131.424240,1760.443237,2794.384766,2030.347168,737.467163,497.728668,NaN,1217.636841,622.909363,2121.218994,209.348557,1794.499878,520.883972,1476.416748,671.314148,57.072010,193.328018,253.251236,1652.108521,1218.171509,504.859314,6965.345215,1385.777222,91.600052,94.772476,81.835182,1380.790405,966.525024,314.093231,20736.521484,422.312195,306.165771,189.920181,19.459246,2010.486938,667.028198,1113.342285,4118.269043,558.514343,245.935379
2020-01-06 00:00:00+00:00,366.808502,1643.927612,720.112366,2606.634277,902.173401,3930.145264,433.420685,176.685226,2735.281006,449.479767,127.608971,1763.849121,2789.492188,1998.841675,718.315308,495.133789,NaN,1191.285400,611.897034,2065.230713,202.433136,1783.327515,508.172424,1419.005615,664.880066,56.194660,190.571976,245.629044,1647.572266,1201.473511,497.041229,6761.932617,1368.974365,91.254692,93.149292,81.877579,1348.766968,974.454712,300.256958,20679.728516,417.895294,300.296448,184.503815,19.038841,2010.303711,662.983521,1131.805298,4057.654297,553.030884,246.963760
2020-01-07 00:00:00+00:00,371.246429,1660.538818,722.601562,2606.891602,907.456177,3940.913086,429.034912,173.615860,2748.449951,451.261261,127.608971,1769.174316,2794.676025,1993.413452,723.480957,497.047943,NaN,1210.148926,622.168518,2047.702271,201.997894,1788.215454,505.465790,1411.521362,655.026367,56.037186,190.774612,246.558594,1665.817139,1204.895142,499.078674,6791.889648,1357.019043,92.521042,92.780373,81.283958,1369.517212,976.239014,299.692200,21562.068359,424.021912,302.990570,183.559677,19.153496,2015.237427,668.749451,1133.124023,4140.621094,562.579773,249.951050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04 00:00:00+00:00,1472.099976,3231.649902,1177.699951,10963.700195,1871.900024,7299.500000,1561.900024,357.250000,5926.549805,1651.900024,503.649994,5096.600098,6787.200195,4833.549805,2757.649902,1785.250000,NaN,1641.800049,758.799988,5683.750000,666.799988,2841.250000,1236.349976,1434.400024,1922.449951,177.029999,506.350006,933.049988,1783.800049,3650.800049,2749.600098,12336.250000,2534.7500

In [26]:
stock_prices = stock_prices.T
stock_prices.head()

Date        2020-01-01 00:00:00+00:00  2020-01-02 00:00:00+00:00  \
ADANIPORTS                 364.348297                 369.654572   
ASIANPAINT                1726.790894                1724.335449   
AXISBANK                   745.452026                 753.666260   
BAJAJ-AUTO                2703.128906                2678.157715   
BAJFINANCE                 935.670654                 948.356201   

Date        2020-01-03 00:00:00+00:00  2020-01-06 00:00:00+00:00  \
ADANIPORTS                 369.027466                 366.808502   
ASIANPAINT                1686.539062                1643.927612   
AXISBANK                   739.726990                 720.112366   
BAJAJ-AUTO                2636.153564                2606.634277   
BAJFINANCE                 932.418274                 902.173401   

Date        2020-01-07 00:00:00+00:00  2020-01-08 00:00:00+00:00  \
ADANIPORTS                 371.246429                 372.066559   
ASIANPAINT                1660.538818                1664.775879   
AXISBANK                   722.601562                 721.356934   
BAJAJ-AUTO                2606.891602                2625.169434   
BAJFINANCE                 907.456177                 912.429688   

Date        2020-01-09 00:00:00+00:00  2020-01-10 00:00:00+00:00  \
ADANIPORTS                 377.999908                 378.241089   
ASIANPAINT                1706.905762                1726.165039   
AXISBANK                   739.627380                 736.839539   
BAJAJ-AUTO                2647.866699                2661.167236   
BAJFINANCE                 937.346741                 935.052063   

Date        2020-01-13 00:00:00+00:00  2020-01-14 00:00:00+00:00  \
ADANIPORTS                 376.601013                 375.491486   
ASIANPAINT                1738.057861                1751.779785   
AXISBANK                   734.201111                 744.655457   
BAJAJ-AUTO                2655.031982                2661.510986   
BAJFINANCE                 943.332825                 953.269714   

Date        2020-01-15 00:00:00+00:00  2020-01-16 00:00:00+00:00  \
ADANIPORTS                 377.035126                 377.179840   
ASIANPAINT                1774.024414                1767.187256   
AXISBANK                   743.062317                 734.101501   
BAJAJ-AUTO                2680.346191                2670.520752   
BAJFINANCE                 956.691833                 959.984070   

Date        2020-01-17 00:00:00+00:00  2020-01-20 00:00:00+00:00  \
ADANIPORTS                 373.947845                 366.663757   
ASIANPAINT                1762.276123                1780.283691   
AXISBANK                   736.690186                 724.045227   
BAJAJ-AUTO                2675.669434                2663.312988   
BAJFINANCE                 969.247620                 955.055603   

Date        2020-01-21 00:00:00+00:00  2020-01-22 00:00:00+00:00  \
ADANIPORTS                 366.567352                 370.088745   
ASIANPAINT                1743.691040                1713.357666   
AXISBANK                   714.486938                 709.508606   
BAJAJ-AUTO                2657.606201                2639.886230   
BAJFINANCE                 951.219482                 952.536438   

Date        2020-01-23 00:00:00+00:00  2020-01-24 00:00:00+00:00  \
ADANIPORTS                 372.259460                 371.198181   
ASIANPAINT                1710.324341                1720.628174   
AXISBANK                   719.863464                 734.201111   
BAJAJ-AUTO                2630.275391                2637.955566   
BAJFINANCE                 951.209473                 961.141418   

Date        2020-01-27 00:00:00+00:00  2020-01-28 00:00:00+00:00  \
ADANIPORTS                 367.821503                 359.813873   
ASIANPAINT                1722.120483                1713.213013   
AXISBANK                   737.038696                 733.952148   
BAJAJ-AUTO                2645.206543          

In [27]:
# putting the data together
prices = stock_prices.iloc[:, -1]
df.index = stock_prices.index
df["returns_2020"] = prices
df.head()

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth,returns_2020
ADANIPORTS,3.101745e+12,1621158.0,0.472,244.328,2.741933e+11,NaN,0.33220,0.472,83.334066
ASIANPAINT,3.145921e+12,1271772.0,-0.245,195.250,3.522494e+11,NaN,0.14421,-0.246,96.242078
AXISBANK,3.621931e+12,7700448.0,0.057,534.043,6.497045e+11,0.01887,0.41144,0.050,56.769951
BAJAJ-AUTO,3.036221e+12,193054.0,0.181,1037.411,4.776608e+11,NaN,0.16761,0.198,99.470692
BAJFINANCE,4.544633e+12,878729.0,0.138,1241.009,3.295906e+11,NaN,0.45287,0.115,64.929790


In [28]:
# checking for null values
df.isna().sum()

marketCap                   1
regularMarketVolume         1
earningsQuarterlyGrowth     2
bookValue                   1
totalRevenue                1
returnOnAssets             39
profitMargins               1
earningsGrowth              3
returns_2020                1
dtype: int64

In [29]:
df["returnOnAssets"] = df["returnOnAssets"].replace(np.nan, 0)

In [30]:
df.dropna(axis = 0, inplace = True)
df.shape

(46, 9)

In [31]:
df.head()

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth,returns_2020
ADANIPORTS,3.101745e+12,1621158.0,0.472,244.328,2.741933e+11,0.00000,0.33220,0.472,83.334066
ASIANPAINT,3.145921e+12,1271772.0,-0.245,195.250,3.522494e+11,0.00000,0.14421,-0.246,96.242078
AXISBANK,3.621931e+12,7700448.0,0.057,534.043,6.497045e+11,0.01887,0.41144,0.050,56.769951
BAJAJ-AUTO,3.036221e+12,193054.0,0.181,1037.411,4.776608e+11,0.00000,0.16761,0.198,99.470692
BAJFINANCE,4.544633e+12,878729.0,0.138,1241.009,3.295906e+11,0.00000,0.45287,0.115,64.929790


In [32]:
# scaling the data using StandardScaler()
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(df)
df_scaled = pd.DataFrame(x, columns = df.columns, index = df.index)
df_scaled

,marketCap,regularMarketVolume,earningsQuarterlyGrowth,bookValue,totalRevenue,returnOnAssets,profitMargins,earningsGrowth,returns_2020
ADANIPORTS,-0.217900,-0.491148,0.997906,-0.391635,-0.603924,-0.367678,1.642563,1.028220,-0.411864
ASIANPAINT,-0.205777,-0.537043,-0.910311,-0.443387,-0.563621,-0.367678,-0.067848,-0.923844,0.045110
AXISBANK,-0.075150,0.307425,-0.106571,-0.086133,-0.410034,0.020410,2.363521,-0.119093,-1.352297
BAJAJ-AUTO,-0.235881,-0.678743,0.223441,0.444663,-0.498867,-0.367678,0.145054,0.283282,0.159411
BAJFINANCE,0.178059,-0.588673,0.109001,0.659355,-0.575321,-0.367678,2.740468,0.057626,-1.063419
BAJAJFINSV,-0.255446,-0.496996,0.007868,-0.250212,-0.140762,-0.367678,-0.731851,0.014126,-0.657487
BPCL,-0.661178,0.504292,-2.209070,-0.465443,1.580219,-0.367678,-0.994977,-2.250595,0.917416
BHARTIARTL,1.462012,-0.190189,3.946727,-0.496122,0.034419,0.734682,-0.776706,3.828536,-0.215925
BRITANNIA,-0.676327,-0.661920,0.021175,-0.476721,-0.657941,-0.367678,-0.206053,0.030438,1.114856
CIPLA,-0.709852,-0.572286,0.228764,-0.300468,-0.611809,-0.367678,0.132317,0.239782,2.000807


Clustering algorithm can now be applied to this data.

# Hierarchical Clustering
